In [2]:
# -*- coding: utf-8 -*-
"""
Exemplo completo de cálculo de isócronas usando OSMnx e alphashape
Autor: Assistente IA
Data: 25 de janeiro de 2026
"""

# 1. Importação das bibliotecas necessárias
import geopandas as gpd
import osmnx as ox
import alphashape
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point, Polygon, MultiPoint
import networkx as nx
from datetime import timedelta
import contextily as ctx
import pandas as pd
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

# Configurações iniciais do OSMnx
ox.config(use_cache=True, log_console=True)

# 2. Leitura dos dados das UBS (substitua pelo seu caminho)
print("=== ETAPA 1: LEITURA DOS DADOS ===")
ubs_gdf = gpd.read_file('../dados/brutos/semplam/ubs_zu_utm.geojson')

# Verificar o sistema de coordenadas
print(f"Sistema de coordenadas original: {ubs_gdf.crs}")
print(f"Número de UBS: {len(ubs_gdf)}")

# 3. Verificação e preparação da projeção
print("\n=== ETAPA 2: PREPARAÇÃO DA PROJEÇÃO ===")
# Verificar se já está projetado (UTM geralmente é projetado)
if ubs_gdf.crs.is_geographic:
    print("Dados estão em coordenadas geográficas. Convertendo para UTM...")
    # Converter para UTM automaticamente
    ubs_gdf = ubs_gdf.to_crs(ubs_gdf.estimate_utm_crs())
    print(f"Novo sistema de coordenadas: {ubs_gdf.crs}")
else:
    print("Dados já estão em sistema projetado (UTM)")

# 4. Definir a área de estudo com base nos pontos das UBS
print("\n=== ETAPA 3: DEFINIÇÃO DA ÁREA DE ESTUDO ===")
# Criar um buffer ao redor de todos os pontos para definir a área
buffer_distance = 5000  # 5km de buffer
study_area = ubs_gdf.unary_union.buffer(buffer_distance)

# Obter os limites da área de estudo
minx, miny, maxx, maxy = study_area.bounds
print(f"Limites da área de estudo: ({minx}, {miny}, {maxx}, {maxy})")

# 5. Download da rede viária com OSMnx
print("\n=== ETAPA 4: DOWNLOAD DA REDE VIÁRIA ===")
# Configurar OSMnx para usar o modo de condução e incluir velocidades
print("Baixando a rede viária da área de estudo...")
G = ox.graph_from_polygon(study_area, network_type='drive', simplify=True)

# Adicionar velocidades e tempos de viagem
print("Adicionando velocidades e calculando tempos de viagem...")
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

# Converter para GeoDataFrame para visualização
nodes, edges = ox.graph_to_gdfs(G)
print(f"Número de nós na rede: {len(nodes)}")
print(f"Número de arestas na rede: {len(edges)}")

# 6. Selecionar um ponto de teste (primeira UBS como exemplo)
print("\n=== ETAPA 5: SELEÇÃO DO PONTO DE TESTE ===")
test_ubs = ubs_gdf.iloc[0]  # Primeira UBS como exemplo
test_point = (test_ubs.geometry.y, test_ubs.geometry.x)  # (lat, lon)

print(f"Ponto de teste selecionado: {test_point}")
print(f"Nome da UBS: {test_ubs.get('nome', 'Sem nome')}")

# 7. Encontrar o nó mais próximo na rede
print("\n=== ETAPA 6: ENCONTRANDO NÓ MAIS PRÓXIMO ===")
orig_node = ox.distance.nearest_nodes(G, X=test_ubs.geometry.x, Y=test_ubs.geometry.y)
print(f"Nó de origem: {orig_node}")

# 8. Calcular isócronas para diferentes tempos de viagem
print("\n=== ETAPA 7: CÁLCULO DAS ISÓCRONAS ===")
# Definir os intervalos de tempo em minutos
travel_times = [5, 10, 15]  # 5, 10 e 15 minutos
iso_colors = ['#FF0000', '#FFA500', '#FFFF00']  # Vermelho, Laranja, Amarelo
iso_labels = ['0-5 min', '5-10 min', '10-15 min']

# Dicionário para armazenar os resultados
isochrones = {}

print("Calculando nós alcançáveis para cada tempo...")
for i, minutes in enumerate(travel_times):
    # Converter minutos para segundos
    travel_time_seconds = minutes * 60
    
    # Calcular subgrafo alcançável
    subgraph = nx.ego_graph(G, orig_node, radius=travel_time_seconds, distance='travel_time')
    
    # Extrair coordenadas dos nós alcançáveis
    nodes_data = []
    for node, data in subgraph.nodes(data=True):
        nodes_data.append({
            'node_id': node,
            'x': data['x'],
            'y': data['y'],
            'geometry': Point(data['x'], data['y'])
        })
    
    # Criar GeoDataFrame com os nós alcançáveis
    reachable_nodes_gdf = gpd.GeoDataFrame(nodes_data, geometry='geometry', crs="EPSG:4326")
    
    # Reprojetar para o mesmo CRS dos dados UTM
    reachable_nodes_gdf = reachable_nodes_gdf.to_crs(ubs_gdf.crs)
    
    isochrones[minutes] = {
        'subgraph': subgraph,
        'nodes_gdf': reachable_nodes_gdf,
        'count': len(subgraph.nodes)
    }
    
    print(f"Tempo: {minutes} min - Nós alcançáveis: {len(subgraph.nodes)}")

# 9. Aplicar alphashape para criar polígonos côncavos
print("\n=== ETAPA 8: APLICANDO ALPHASHAPE ===")
isochrone_polygons = {}

# Parâmetro alpha - ajustar conforme a densidade dos pontos
# Valores menores = polígonos mais detalhados (côncavos)
# Valores maiores = polígonos mais simplificados (próximos de convex hull)
alpha_param = 100  # Ajuste este valor conforme necessário

for minutes in travel_times:
    print(f"\nProcessando isócrona de {minutes} minutos...")
    
    nodes_gdf = isochrones[minutes]['nodes_gdf']
    
    if len(nodes_gdf) < 4:
        print(f"Aviso: Poucos pontos ({len(nodes_gdf)}) para criar alpha shape. Usando convex hull.")
        points = [(point.x, point.y) for point in nodes_gdf.geometry]
        if len(points) >= 3:
            polygon = Polygon(points).convex_hull
            isochrone_polygons[minutes] = polygon
        else:
            print(f"Erro: Número insuficiente de pontos para {minutes} minutos")
            continue
    else:
        # Extrair coordenadas dos pontos
        points = [(point.x, point.y) for point in nodes_gdf.geometry]
        
        try:
            print(f"Tentando alpha = {alpha_param}...")
            alpha_shape = alphashape.alphashape(points, alpha_param)
            
            # Se o resultado não for um polígono válido, tentar ajustar alpha
            if not isinstance(alpha_shape, Polygon):
                print("Alpha shape não produziu polígono válido. Tentando ajustar parâmetro...")
                
                # Tentar encontrar um bom valor de alpha automaticamente
                optimal_alpha = None
                for test_alpha in [50, 100, 200, 500, 1000]:
                    try:
                        test_shape = alphashape.alphashape(points, test_alpha)
                        if isinstance(test_shape, Polygon) and not test_shape.is_empty:
                            area = test_shape.area
                            if area > 0:  # Verificar se tem área válida
                                optimal_alpha = test_alpha
                                alpha_shape = test_shape
                                print(f"Encontrado alpha válido: {test_alpha}")
                                break
                    except:
                        continue
                
                if optimal_alpha is None:
                    print("Nenhum alpha válido encontrado. Usando convex hull como fallback.")
                    alpha_shape = MultiPoint(points).convex_hull
            
            isochrone_polygons[minutes] = alpha_shape
            print(f"Polígono criado com sucesso! Área: {alpha_shape.area:.2f}")
            
        except Exception as e:
            print(f"Erro ao criar alpha shape: {e}")
            print("Usando convex hull como fallback.")
            alpha_shape = MultiPoint(points).convex_hull
            isochrone_polygons[minutes] = alpha_shape

# 10. Visualização dos resultados
print("\n=== ETAPA 9: VISUALIZAÇÃO DOS RESULTADOS ===")

# Criar figura e eixos
fig, ax = plt.subplots(figsize=(15, 12))

# Plotar a rede viária como contexto (opcional - pode ser pesado)
# print("Plotando rede viária de fundo...")
# edges_plot = edges.to_crs(ubs_gdf.crs)
# edges_plot.plot(ax=ax, linewidth=0.5, color='gray', alpha=0.3, zorder=1)

# Plotar os polígonos das isócronas
print("Plotando polígonos das isócronas...")
for i, minutes in enumerate(travel_times):
    if minutes in isochrone_polygons:
        polygon = isochrone_polygons[minutes]
        
        if isinstance(polygon, Polygon) and not polygon.is_empty:
            # Criar GeoSeries para plotagem
            poly_gdf = gpd.GeoSeries([polygon], crs=ubs_gdf.crs)
            poly_gdf.plot(ax=ax, color=iso_colors[i], alpha=0.3, edgecolor=iso_colors[i], 
                         linewidth=2, label=f'{iso_labels[i]}', zorder=3)
            
            # Calcular centróide para rótulo
            centroid = polygon.centroid
            ax.annotate(f'{iso_labels[i]}', (centroid.x, centroid.y),
                       fontsize=10, fontweight='bold', ha='center',
                       bbox=dict(boxstyle="round,pad=0.3", fc="white", ec=iso_colors[i], alpha=0.8))

# Plotar os nós alcançáveis para cada tempo
print("Plotando nós alcançáveis...")
for i, minutes in enumerate(travel_times):
    if minutes in isochrones:
        nodes_gdf = isochrones[minutes]['nodes_gdf']
        nodes_gdf.plot(ax=ax, color=iso_colors[i], alpha=0.6, markersize=8, 
                      label=f'Nós {minutes}min', zorder=4)

# Plotar todas as UBS
print("Plotando todas as UBS...")
ubs_gdf.plot(ax=ax, color='blue', marker='o', markersize=50, alpha=0.8, 
            label='Todas as UBS', edgecolor='black', linewidth=1, zorder=5)

# Destacar a UBS de teste
test_ubs_gdf = gpd.GeoDataFrame([test_ubs], geometry='geometry', crs=ubs_gdf.crs)
test_ubs_gdf.plot(ax=ax, color='red', marker='*', markersize=200, alpha=1.0, 
                 label='UBS Teste', edgecolor='yellow', linewidth=2, zorder=6)

# Adicionar contexto de mapa (requer internet)
print("Adicionando mapa de fundo...")
try:
    ctx.add_basemap(ax, crs=ubs_gdf.crs, source=ctx.providers.CartoDB.Positron, zorder=0)
except Exception as e:
    print(f"Erro ao adicionar mapa de fundo: {e}")
    print("Continuando sem mapa de fundo.")

# Configurar o plot
ax.set_title('Isócronas de Tempo de Viagem a partir da UBS de Teste', fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude (UTM)', fontsize=12)
ax.set_ylabel('Latitude (UTM)', fontsize=12)
ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
ax.grid(True, alpha=0.3)

# Ajustar limites para focar na área de interesse
buffer_plot = 2000  # 2km de buffer para visualização
plot_area = test_ubs.geometry.buffer(buffer_plot)
minx, miny, maxx, maxy = plot_area.bounds
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

plt.tight_layout()
plt.savefig('isocronas_ubs_teste.png', dpi=300, bbox_inches='tight')
print("\nFigura salva como 'isocronas_ubs_teste.png'")

# 11. Salvar resultados para análise posterior
print("\n=== ETAPA 10: SALVANDO RESULTADOS ===")

# Criar GeoDataFrame com os polígonos das isócronas
isochrone_data = []
for minutes in travel_times:
    if minutes in isochrone_polygons:
        polygon = isochrone_polygons[minutes]
        if isinstance(polygon, Polygon) and not polygon.is_empty:
            isochrone_data.append({
                'tempo_min': minutes,
                'intervalo': f'0-{minutes} min' if minutes == 5 else f'{minutes-5}-{minutes} min',
                'area_m2': polygon.area,
                'num_nos': isochrones[minutes]['count'],
                'geometry': polygon
            })

if isochrone_data:
    isochrones_gdf = gpd.GeoDataFrame(isochrone_data, geometry='geometry', crs=ubs_gdf.crs)
    isochrones_gdf.to_file('isocronas_resultado.geojson', driver='GeoJSON')
    print("Resultados salvos em 'isocronas_resultado.geojson'")

# 12. Análise estatística rápida
print("\n=== ANÁLISE ESTATÍSTICA ===")
print("\nResumo das isócronas:")
for minutes in travel_times:
    if minutes in isochrones:
        nodes_count = isochrones[minutes]['count']
        if minutes in isochrone_polygons and isinstance(isochrone_polygons[minutes], Polygon):
            area = isochrone_polygons[minutes].area
            print(f"{minutes} minutos: {nodes_count} nós alcançáveis, Área: {area/1e6:.2f} km²")
        else:
            print(f"{minutes} minutos: {nodes_count} nós alcançáveis")

# Mostrar o plot
plt.show()

print("\n=== PROCESSO CONCLUÍDO COM SUCESSO ===")
print("O exemplo demonstrou todo o fluxo de trabalho para cálculo de isócronas:")
print("1. Leitura e preparação dos dados das UBS")
print("2. Download da rede viária com OSMnx")
print("3. Cálculo dos nós alcançáveis para diferentes tempos")
print("4. Aplicação do alphashape para polígonos mais realistas")
print("5. Visualização e salvamento dos resultados")

print("\nPróximos passos sugeridos:")
print("- Ajustar o parâmetro alpha conforme a densidade urbana")
print("- Repetir o processo para todas as 75 UBS usando um loop")
print("- Calcular estatísticas de cobertura populacional dentro das isócronas")
print("- Comparar os resultados com convex hull para ver a diferença")

AttributeError: module 'osmnx' has no attribute 'config'